In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Importing all the csv's

train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")

sample = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
train.loss.describe()

In [ ]:
#X and y for the training set

X = train.drop(['id','loss'], axis=1).values
y = train['loss'].values

In [ ]:
#Initializing for K-folds

train_oof = np.zeros((train.shape[0],1))
test_preds = np.zeros((test.shape[0],1))

target = train['loss'].values.reshape(-1,1)

In [ ]:
#X for the test set

X_test = test.drop('id', axis=1).values

In [ ]:
#XGB training and Kfold validation 

from sklearn.model_selection import KFold
from xgboost import XGBRegressor

n_splits = 20
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = target[train_index]
    
    val_features = X[val_index]
    val_target = target[val_index]
    
    model = XGBRegressor( learning_rate=0.05, n_estimators=500, subsample=1.0,alpha=0, tree_method = 'gpu_hist')
    model.fit(train_features, train_target)
    val_pred = model.predict(val_features)
    train_oof[val_index,0] = val_pred.flatten()
    test_preds[:,0] += model.predict(X_test).flatten()/n_splits


In [ ]:
#Saving results

np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [ ]:
#Checking root mean squared error

from sklearn.metrics import mean_squared_error
RMSE = (mean_squared_error(target, train_oof))**(1/2)

In [ ]:
RMSE

In [ ]:
submission = test['id']
df = pd.DataFrame(submission, columns=['id'])
df['loss'] = test_preds

In [ ]:
df.head()

In [ ]:
#Exporting to csv

df.to_csv("submission.csv", index = False)